In [2]:
from he import *

import ipywidgets as widgets
import matplotlib.pyplot as plt

class HeDemo:
    he = HarmonicEntropy(N=100, a=7, limit=1200, s=17, verbose=0)
    he3 = HarmonicEntropy(N=100, a=7, limit=1200, s=17, he3=True, verbose=0)

    figure = None
    entropy1 = None
    entropy2 = None

    def update(self, N=None, limit=None, alpha=None, spread=None, weight=None):
        # global entropy
        # global entropy_triads
        # global im
        global plt

        updateBasis = False
        if N is not None and self.he.N != N:
            updateBasis = True

        cents = None
        if limit is not None:
            cents = ratio_to_cents(limit)
            if self.he.limit != cents:
                updateBasis = True

        self.he.update(spread, N, None, cents, alpha, weight)
        self.he3.update(spread, N, None, cents, alpha, weight)

        if updateBasis:
            self.he.prepareBasis()
            self.he3.prepare3heBasis()

        self.figure, [ self.entropy1,
                       self.entropy2 ] = plt.subplots(ncols=2, figsize=(15, 5), gridspec_kw={'width_ratios':[5,3], 'height_ratios':[1]})

        self.he.calculate()
        self.he3.calculate()

        self.entropy1.plot(self.he.x_axis, self.he.Entropy)
        y_max = self.he.Entropy.max()
        if y_max < 6:
            y_max = 6

        self.entropy1.set_ylim((0, y_max))

        self.entropy2.imshow(self.he3.Entropy, cmap="inferno", aspect="equal", origin="lower")
        self.entropy2.set_axis_off()


demo = HeDemo()

plot_controls = widgets.interact(demo.update,
                N=[10,25,50,100,200,500],
                limit=[2,3,4,5,6,7,8],
                alpha=(1.0, 10.0),
                weight=["default", "lencf", "lenmaxcf", "sumcf"],
                spread=(1.0, 20.0)
                )

plt.show()



interactive(children=(Dropdown(description='N', options=(10, 25, 50, 100, 200, 500), value=None), Dropdown(des…